In [1]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import albumentations

In [2]:
# 確認 GPU 狀態
print("CUDA available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())


CUDA available: True
Number of GPUs: 1


In [3]:
# Step 3: 建立或載入 YOLO 模型
# --------------------------
model = YOLO("yolov11/yolo11m.pt")  # 載入預訓練權重
model.info()


YOLO11m summary: 409 layers, 20,114,688 parameters, 0 gradients, 68.5 GFLOPs


(409, 20114688, 0, 68.52838399999999)

In [4]:
results = model.train(
    data="yolov9/dataset.yaml",    # 你的資料集設定檔
    epochs=300,
    batch=16,
    lr0=0.001,
    weight_decay=0.002,
    cls=1.2,
    box=9.0,
    optimizer="SGD",
    plots=True,
    verbose=True,
    

    # # ------------------------------
    # # 以下為調整後的資料增強參數設定
    # # ------------------------------
    
    # # 幾何轉換
    # degrees=5.0,       # 小幅度旋轉，避免破壞結構
    # translate=0.1,     # 平移
    # scale=0.1,         # 縮放
    # shear=0.0,         # 不做剪切
    # perspective=0.0,   # 不做透視
    
    # # 翻轉
    # flipud=0.0,        # 垂直翻轉機率 0%，避免肺部上下顛倒
    # fliplr=0.3,        # 水平翻轉機率 30%，平衡增強與臨床合理性
    
    # # 強度/對比度調整
    # hsv_h=0.0,         # 不做色相調整
    # hsv_s=0.0,         # 不做飽和度調整
    # hsv_v=0.1,         # 亮度微調10%，避免過度改變HU
    
    # # 複合增強
    # mixup=0.1,         # 混合影像機率降低至10%
    # mosaic=0.1,        # 拼接影像機率降低至10%
)


New https://pypi.org/project/ultralytics/8.3.55 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.25  Python-3.12.4 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11264MiB)
engine\trainer: task=detect, mode=train, model=yolov11/yolo11m.pt, data=yolov9/dataset.yaml, epochs=300, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train13, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=F

c:\Users\UM\anaconda3\envs\pytorch2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-29 23:50:47,653	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-12-29 23:50:48,102	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    111872  ultralytics.nn.modules.block.C3k2            [128, 256, 1, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  1    444928  ultralytics.nn.modules.block.C3k2            [256, 512, 1, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  1   1380352  ultralytics.nn.modules.block.C3k2            [512, 512, 1, True]           
  7                  -1  1   2360320  ultralytics

train: Scanning D:\luna\luna_dataset\train.cache... 963 images, 0 backgrounds, 0 corrupt: 100%|██████████| 963/963 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

    240/300      8.51G     0.6607     0.7369     0.7801          4        640: 100%|██████████| 61/61 [00:49<00:00,  1.24it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:05<00:00,  1.39s/it]

                   all        118        118      0.768       0.73      0.782      0.448
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 140, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



240 epochs completed in 3.693 hours.
Optimizer stripped from runs\detect\train13\weights\last.pt, 40.5MB
Optimizer stripped from runs\detect\train13\weights\best.pt, 40.5MB

Validating runs\detect\train13\weights\best.pt...
Ultralytics 8.3.25  Python-3.12.4 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11264MiB)
YOLO11m summary (fused): 303 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.38it/s]


                   all        118        118      0.779      0.771      0.816      0.475
Speed: 0.5ms preprocess, 13.7ms inference, 0.0ms loss, 3.7ms postprocess per image
Results saved to runs\detect\train13


In [5]:
val_results = model.val()
train_results = model.val(data="yolov9/dataset.yaml", split='train')
test_results = model.val(data="yolov9/dataset.yaml", split='test')

print("Validation Results:", val_results)
print("Train Results:", train_results)
print("Test Results:", test_results)

Ultralytics 8.3.25  Python-3.12.4 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11264MiB)


YOLO11m summary (fused): 303 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs


val: Scanning D:\luna\luna_dataset\val.cache... 118 images, 0 backgrounds, 0 corrupt: 100%|██████████| 118/118 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:12<00:00,  1.50s/it]


                   all        118        118       0.79       0.78      0.827       0.48
Speed: 0.8ms preprocess, 14.3ms inference, 0.0ms loss, 3.9ms postprocess per image
Results saved to runs\detect\train1317
Ultralytics 8.3.25  Python-3.12.4 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11264MiB)


val: Scanning D:\luna\luna_dataset\train.cache... 963 images, 0 backgrounds, 0 corrupt: 100%|██████████| 963/963 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 61/61 [00:20<00:00,  2.94it/s]


                   all        963        963      0.979      0.988      0.994      0.843
Speed: 0.6ms preprocess, 13.1ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs\detect\train1318
Ultralytics 8.3.25  Python-3.12.4 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11264MiB)


val: Scanning D:\luna\luna_dataset\test.cache... 105 images, 0 backgrounds, 0 corrupt: 100%|██████████| 105/105 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:12<00:00,  1.75s/it]


                   all        105        105      0.821       0.79      0.826      0.489
Speed: 1.2ms preprocess, 14.9ms inference, 0.0ms loss, 4.0ms postprocess per image
Results saved to runs\detect\train1319
Validation Results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002732EF33920>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.0